<h1><center>NHL DATA SCRAPE AND VISUALIZATION</center></h1>
<h2><center>A Learning Project</center></h2>

The following is an attempt to learn pandas, numpy, and beautifulsoup in order to create a repository of data scrapped from the web, clean it, and then process it to draw and test assumptions against that data. This will utilize the NHL API along with the NHL Play by Play sheet. One of the reasons to use a Jupyter Notebook is to avoid multiple calls to the mentioned websites but still continue to work with the data they provide.

First we import the required python libraries. 
- NHLget is a self made module which accesses the NHL API and the NHL Play by Play pages. It has five functions, three are internal to the module and the other two can be accessed. One of the outward facing functions returns a list of gameIDs for a provided time period. The other returns play information for a speific game. The three inner functions work with the later. One gets the game information from the NHL API, the second gets additional game information from the NHL Game Report, and the final one combines those two and then returns the game information.

In [1]:
import NHLget

In [2]:
all_plays = NHLget.get_player_on_ice_info('2021020001')

['6', '1', 'EV', ['00:18', '19:42'], 'HIT', 'TBL #18 PALAT HIT PIT #77 CARTER, Off. Zone', {'C': '77', 'R': '17', 'L': '43', 'D1': '8', 'D2': '58', 'G': '35'}, {'C': '21', 'R': '86', 'L': '18', 'D1': '27', 'D2': '81', 'G': '88'}, {'player': {'id': 8476292, 'fullName': 'Ondrej Palat', 'link': '/api/v1/people/8476292'}, 'playerType': 'Hitter'}, {'player': {'id': 8470604, 'fullName': 'Jeff Carter', 'link': '/api/v1/people/8470604'}, 'playerType': 'Hittee'}, '2021-10-12T23:44:24Z', {'x': 46.0, 'y': 40.0}]


In [ ]:
print(all_plays[6])